#### Setup

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf

In [ ]:
# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

#### Load Data

In [ ]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
data = [train, test]

#### Clean Data

In [ ]:
# Null Values
for df in data:
    mean_age = df['Age'].mean()
    df['Age'] = df['Age'].fillna(mean_age)
    df['Age'] = df['Age'].astype(int)
    mode_embarked = df['Embarked'].mode()[0]
    df['Embarked'] = df['Embarked'].fillna(mode_embarked)
    df['Embarked'] = df['Embarked'].astype(str)

#drop string columns
for df in data:
    df.drop(columns = ['Name', 'Ticket', 'Cabin'], inplace = True)

# Feature Engineering
for df in data:
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = 1
    df.loc[df['FamilySize'] > 1, 'IsAlone'] = 0
    df['Fare_per_person'] = df['Fare'] / df['FamilySize']

for df in data:
    print (df.head())



#### Data Preprocessing

In [ ]:
cat_cols = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'FamilySize']
num_cols = ['Age', 'Fare']

def preprocess(df, cat_cols, num_cols):
    encoder = OneHotEncoder(sparse_output = False)
    scaler = StandardScaler()
    
    #encode columns
    encoded_cols = encoder.fit_transform(df[cat_cols])
    encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(cat_cols))
    df = pd.concat([df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)
    
    #scale columns
    scaled_cols = scaler.fit_transform(df[num_cols])
    scaled_df = pd.DataFrame(scaled_cols, columns=num_cols)
    df[num_cols] = scaled_df
    
    #drop old columns
    df.drop(columns=cat_cols + num_cols, inplace=True)
    
    return df
    
test = preprocess(test, cat_cols, num_cols)
train = preprocess(train, cat_cols, num_cols)

#manually add missing column because I'm not sure how to do it automatically
train['Parch_9'] = 0
           


In [ ]:
print(test.columns)
print(train.columns)
print(test.shape)
print(train.shape)

In [ ]:
from sklearn.model_selection import train_test_split
#create train test split
X = train.drop(columns = 'Survived')
y = train['Survived']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2) 


In [17]:
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from kerastuner import HyperModel
from kerastuner.tuners import Hyperband

# Hypermodel
class TitanicHyperModel(HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Input(shape=(X_train.shape[1],)))
        model.add(layers.BatchNormalization())
        
        for i in range(hp.Int('num_layers', 1, 5)):
            units = hp.Int('units_' + str(i), 
                           min_value=64, 
                           max_value=1024, 
                           step=32)
            activation = hp.Choice('activation_' + str(i), ['relu', 'leaky_relu'])

            kernel_regularizer = regularizers.l2(hp.Float('l2_regularization_' + str(i),
                                                          min_value=1e-7,
                                                          max_value=0.1,
                                                          sampling='LOG'))
            model.add(layers.Dense(units=units,
                                   activation=activation,
                                   kernel_regularizer=kernel_regularizer))
            model.add(layers.BatchNormalization())
            model.add(layers.Dropout(hp.Float('dropout_' + str(i),
                                              min_value=0.1,
                                              max_value=0.5,
                                              step=0.1)))
        model.add(layers.Dense(1, activation='sigmoid'))

        optimizer = hp.Choice('optimizer', ['adam', 'rmsprop'])
        if optimizer == 'adam':
            optimizer = keras.optimizers.Adam(
                learning_rate=hp.Float('learning_rate',
                                       min_value=0.001,
                                       max_value=0.01,
                                       sampling='LOG')
            )
        else:
            optimizer = keras.optimizers.RMSprop(
                learning_rate=hp.Float('learning_rate',
                                       min_value=0.0001,
                                       max_value=0.01,
                                       sampling='LOG')
            )

        model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        return model

    def get_build_config(self):
        return {}

    def build_from_config(self, config):
        return self.build(config)

tuner = Hyperband(
    TitanicHyperModel(),
    objective='val_loss',
    max_epochs=100,
    factor=3,
    directory='my_dir',
    project_name='Titanic Survival Prediction'
)

#*********************#
MY_PATIENCE = 10
MY_EPOCHS = 100
MY_MIN_DELTA = 1e-4
#*********************#

tuner.search(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=MY_EPOCHS,
    callbacks=[keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=MY_PATIENCE,
        min_delta=MY_MIN_DELTA,
        restore_best_weights=True
    )],
    verbose=1
)

best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_hyperparameters.values)
print(best_model.summary())

best_model.compile(
    optimizer=keras.optimizers.Adam(
        learning_rate=best_hyperparameters.get('learning_rate')
    ),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
best_model.evaluate(X_valid, y_valid)

Trial 157 Complete [00h 00m 06s]
val_loss: 0.6146777868270874

Best val_loss So Far: 0.4133809804916382
Total elapsed time: 00h 13m 48s

Search: Running Trial #158

Value             |Best Value So Far |Hyperparameter
2                 |2                 |num_layers
896               |416               |units_0
leaky_relu        |leaky_relu        |activation_0
0.00046307        |8.0308e-07        |l2_regularization_0
0.4               |0.3               |dropout_0
adam              |rmsprop           |optimizer
0.0012583         |0.0038783         |learning_rate
352               |288               |units_1
relu              |leaky_relu        |activation_1
2.294e-06         |4.2044e-07        |l2_regularization_1
0.3               |0.5               |dropout_1
704               |352               |units_2
leaky_relu        |relu              |activation_2
0.0067845         |0.0052908         |l2_regularization_2
0.5               |0.2               |dropout_2
960               |256  

KeyboardInterrupt: 

#### Plots of Model

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

history = best_model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=MY_EPOCHS,  # Ensure you specify the number of epochs
    callbacks=[keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=MY_PATIENCE,
        min_delta=MY_MIN_DELTA,
        restore_best_weights=True
    )],
    verbose=1
)

# Plotting the training and validation accuracy
sns.set_style('whitegrid')
sns.set_context('notebook')

plt.figure(figsize=(10, 6))
epochs_range = range(1, len(history.history['accuracy']) + 1)
sns.lineplot(x=epochs_range, y=history.history['accuracy'], label='Train Accuracy', marker='o')
sns.lineplot(x=epochs_range, y=history.history['val_accuracy'], label='Validation Accuracy', marker='o')

plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.title('Training and Validation Accuracy over Epochs', fontsize=16)
plt.legend(fontsize=12)
plt.show()

In [ ]:
#Plot Training and validation loss
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss vs val_loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Assuming y_test contains the true labels for the test set
y_pred_prob = best_model.predict(X_valid)

# Convert probabilities to class labels using a threshold (e.g., 0.5)
y_pred_class = (y_pred_prob > 0.5).astype("int32")

# Compute ROC curve and ROC area
fpr, tpr, thresholds = roc_curve(y_valid, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred_class)

# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

# Example: Adjust threshold to 0.6
new_threshold = 0.6
y_pred_class_adjusted = (y_pred_prob > new_threshold).astype("int32")

# Compute confusion matrix for adjusted threshold
cm_adjusted = confusion_matrix(y_test, y_pred_class_adjusted)

# Plot confusion matrix for adjusted threshold
disp_adjusted = ConfusionMatrixDisplay(confusion_matrix=cm_adjusted, display_labels=[0, 1])
disp_adjusted.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Adjusted Threshold)')
plt.show()

In [ ]:
predictions = best_model.predict_classes(test)

# Convert predictions to binary (0 or 1)
#binary_predictions = (predictions > 0.5).astype(int)

# Convert predictions to DataFrame
predictions_df = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    #'Survived': binary_predictions.flatten()
    'Survived': predictions.flatten()
})

# Save predictions to CSV
predictions_df.to_csv('Titanic_NN_predictions.csv', index=False)